Скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

сделать разбиение набора данных не тренировочную и тестовую выборки.

сделать feature engineering на ваше усмотрение (допускается свобода выбора методов).

провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями.

в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей.

In [1]:
%matplotlib inline

import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.models import SoloModel
from sklift.models import ClassTransformation
from sklift.models import TwoModels
from catboost import CatBoostClassifier

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data.csv')
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


In [3]:
for col in df.columns:
    print(f'{col}: {df[col].unique()}')

recency: [10  6  7  9  2  1  5  4  3 11 12  8]
history: [142.44 329.08 180.65 ... 519.69 105.54 552.94]
used_discount: [1 0]
used_bogo: [0 1]
zip_code: ['Surburban' 'Rural' 'Urban']
is_referral: [0 1]
channel: ['Phone' 'Web' 'Multichannel']
offer: ['Buy One Get One' 'No Offer' 'Discount']
conversion: [0 1]


In [4]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)

df['treatment'] = df['treatment'].replace({'No Offer': 0, 'Buy One Get One': 1, 'Discount': 1})

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1), df['target'], train_size=0.7, random_state=1)

treat_train = X_train['treatment']
treat_test = X_test['treatment']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((44800, 8), (19200, 8), (44800,), (19200,))

In [6]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['history']
cat_cols = ['zip_code', 'channel', 'recency']
base_cols = ['used_discount', 'used_bogo', 'is_referral', 'treatment']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [7]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

X_train = feature_processing.fit_transform(X_train)
X_test = feature_processing.fit_transform(X_test)

In [8]:
treatment_model = CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True)
control_model = CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True)

score = pd.DataFrame(columns=['model', 'uplift@10%', 'uplift@20%'])

models = [SoloModel(treatment_model),
          ClassTransformation(treatment_model),
          TwoModels(estimator_trmnt=treatment_model, estimator_ctrl=control_model, method='vanilla')]

for model in models:
    md = model
    md.fit(X_train, y_train, treat_train)
    uplift = md.predict(X_test)
    
    score_10 = uplift_at_k(y_true=y_test, uplift=uplift, treatment=treat_test, strategy='by_group', k=0.1)
    score_20 = uplift_at_k(y_true=y_test, uplift=uplift, treatment=treat_test, strategy='by_group', k=0.2)
    
    score = score.append({'model': type(md).__name__, 'uplift@10%': score_10, 'uplift@20%': score_20}, ignore_index=True)

In [9]:
score

,model,uplift@10%,uplift@20%
0,SoloModel,0.106057,0.076832
1,ClassTransformation,0.287656,0.213937
2,TwoModels,0.064979,0.071987
